In [60]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import GroupShuffleSplit # used for splitting data 
from torch_geometric.data import Data


In [84]:
data_dir = os.path.normpath(r'D:\Projects\cs224-multimodal-recommender-system\processed_data\nowplaying')


train = pd.read_parquet(
    os.path.join(data_dir, 'session_candidates_train.parquet')
)

test = pd.read_parquet(
    os.path.join(data_dir, 'session_candidates_test.parquet')
)

features = pd.read_parquet(
    os.path.join(data_dir, 'Lyrics_HSP-L_Nowplay_Data.parquet')
)

track_map = pd.read_parquet(
    os.path.join(data_dir, 'track_id_map.parquet')
).set_index(['Artist', 'Title'])


In [93]:
track_features = (features
                  .drop_duplicates(subset= ['Artist', 'Title'], keep = 'first')
                  .drop(columns = ['user id', 'source of the tweet', 'track title', 'artist name', ])
                  )
print('Initial number of tracks:', track_features.shape[0])
track_features = track_features.merge(track_map, on = ['Artist', 'Title'], how = 'inner')
track_features = (track_features
                  .set_index('track_id')
                  .lyrics_embedding
                  .apply(lambda x: pd.Series(x))
                  )

print('Number of tracks post-inner join:', track_features.shape[0])
print('Track features')
print(79*'-')
display(track_features.head())

train_user_listening_history = train.loc[:, ['user', 'past_interactions']]
print('Train: User listening history')
print(79*'-')
display(train_user_listening_history.head())


train_user_item_interactions = train.loc[:, ['user', 'positive_song_id']]
print('Train: User-item interactions')
print(79*'-')
display(train_user_item_interactions.head())

train_user_item_all_interactions = train.loc[: , ['user', 'candidates']].explode('candidates').rename(columns = {'candidates': 'song_id'})
print('Train: User-item negative + positive interactions')
print(79*'-')
display(train_user_item_all_interactions.head())

train_track_features = track_features.loc[train_user_item_all_interactions.song_id.unique(), :]



test_user_listening_history = test.loc[:, ['user', 'past_interactions']]
print('Test: User listening history')
print(79*'-')
display(test_user_listening_history.head())


test_user_item_interactions = test.loc[:, ['user', 'positive_song_id']]
print('Test: User-item interactions')
print(79*'-')
display(test_user_item_interactions.head())

test_user_item_all_interactions = test.loc[: , ['user', 'candidates']].explode('candidates').rename(columns = {'candidates': 'song_id'})
print('Test: User-item negative + positive interactions')
print(79*'-')
display(test_user_item_all_interactions.head())

test_track_features = track_features.loc[test_user_item_all_interactions.song_id.unique(), :]

# positive interaction tuples
train_positive_interactions_set = set(
    train_user_item_interactions
    .drop_duplicates()
    .apply(lambda x: (x['user'], x['positive_song_id']), axis = 1)
    .to_list()
)
test_positive_interactions_set = set(
    test_user_item_interactions
    .drop_duplicates()
    .apply(lambda x: (x['user'], x['positive_song_id']), axis = 1)
    .to_list()
)



Initial number of tracks: 2471
Number of tracks post-inner join: 2155
Track features
-------------------------------------------------------------------------------


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
track_id,,,,,,,,,,,,,,,,,,,,,
1,-0.016555,0.016258,0.032462,-0.065711,0.035892,-0.030516,0.044030,0.046941,0.031009,-0.051097,...,0.137975,-0.032969,-0.178082,0.032461,0.081872,0.018873,0.134802,-0.184255,-0.020970,-0.009385
2,-0.043866,0.079759,0.031227,-0.009023,0.198229,-0.117961,0.023687,0.126027,0.002055,-0.055694,...,0.082887,-0.094865,-0.103203,0.029579,0.168657,0.054824,0.156257,0.036643,-0.045506,-0.051694
3,0.040213,0.071726,-0.022398,-0.148623,0.250008,-0.033887,0.026806,0.242937,0.030847,0.016596,...,0.093019,-0.006108,-0.108450,0.160098,0.071841,0.118777,0.233991,0.042564,-0.028533,-0.093544
4,-0.009909,-0.046708,0.058077,0.009227,0.064153,-0.003633,0.027486,0.102278,0.117365,-0.024824,...,0.100890,-0.193991,-0.133274,0.108100,0.134432,0.114251,0.241544,-0.116317,-0.074772,-0.075406
5,0.003207,0.011662,0.044229,-0.107282,0.164013,-0.027828,0.017900,0.089839,0.104875,-0.010808,...,0.083276,-0.045836,-0.046923,0.081197,0.055439,0.086655,0.196141,-0.123199,-0.025011,0.013837


Train: User listening history
-------------------------------------------------------------------------------


,user,past_interactions
0,1001b22c8e4adeb77ef10481ad06ff9c35006cb3,"[96, 46, 46, 46, 46, 232, 325, 232, 125, 325]"
1,1001b22c8e4adeb77ef10481ad06ff9c35006cb3,"[46, 46, 46, 46, 232, 325, 232, 125, 325, 125]"
2,1001b22c8e4adeb77ef10481ad06ff9c35006cb3,"[46, 46, 46, 232, 325, 232, 125, 325, 125, 232]"
3,1001b22c8e4adeb77ef10481ad06ff9c35006cb3,"[46, 46, 232, 325, 232, 125, 325, 125, 232, 46]"
4,1001b22c8e4adeb77ef10481ad06ff9c35006cb3,"[46, 232, 325, 232, 125, 325, 125, 232, 46, 96]"


Train: User-item interactions
-------------------------------------------------------------------------------


...

,user,past_interactions
0,103aaf06662cc01fe8b1ddaa39e41f59a8332832,"[595, 84, 152, 614, 688, 48, 34, 191, 1362, 171]"
1,103aaf06662cc01fe8b1ddaa39e41f59a8332832,"[84, 152, 614, 688, 48, 34, 191, 1362, 171, 111]"
2,103aaf06662cc01fe8b1ddaa39e41f59a8332832,"[152, 614, 688, 48, 34, 191, 1362, 171, 111, 196]"
3,103aaf06662cc01fe8b1ddaa39e41f59a8332832,"[614, 688, 48, 34, 191, 1362, 171, 111, 196, 152]"
4,103aaf06662cc01fe8b1ddaa39e41f59a8332832,"[688, 48, 34, 191, 1362, 171, 111, 196, 152, 643]"


Test: User-item interactions
-------------------------------------------------------------------------------


,user,positive_song_id
0,103aaf06662cc01fe8b1ddaa39e41f59a8332832,111
1,103aaf06662cc01fe8b1ddaa39e41f59a8332832,196
2,103aaf06662cc01fe8b1ddaa39e41f59a8332832,152
3,103aaf06662cc01fe8b1ddaa39e41f59a8332832,643
4,103aaf06662cc01fe8b1ddaa39e41f59a8332832,144


Test: User-item negative + positive interactions
-------------------------------------------------------------------------------


,user,song_id
0,103aaf06662cc01fe8b1ddaa39e41f59a8332832,427
0,103aaf06662cc01fe8b1ddaa39e41f59a8332832,46
0,103aaf06662cc01fe8b1ddaa39e41f59a8332832,995
0,103aaf06662cc01fe8b1ddaa39e41f59a8332832,1350
0,103aaf06662cc01fe8b1ddaa39e41f59a8332832,1548


In [96]:
import torch

def prepare_graph(user_item_interactions, user_item_all_interactions , item_features, positive_interactions_set):
    # item feature manipulation 
    # get user id and item id
    user_ids = list(set([ui for ui, ii in user_item_all_interactions]))
    item_ids = item_features.index
    
    # create map for users and items
    user_id_map = {uid: idx for idx, uid in enumerate(user_ids)}
    num_users = len(user_ids)
    item_id_map = {iid: idx + num_users for idx, iid in enumerate(item_ids)}
    print('Number of users: ', num_users)
    num_items = len(item_ids)
    print('Number of items: ', num_items)
    
    
    
    num_features = item_features.shape[1]
    num_nodes = num_users + num_items
    x = torch.zeros(num_nodes, num_features)
    for iid, idx in item_id_map.items():
        x[idx] = torch.tensor(item_features.loc[iid, :].values, dtype = torch.float)
    
    edge_index = []
    for uid, iid in user_item_interactions:
        u_idx = user_id_map[uid]
        i_idx = item_id_map[iid]
        # add undirected edges
        # TODO: Undirected edges is a major caveat of this architecture 
        #  However it is necessary for us to consider this style of architecture because
        #  otherwise the maximum depth of the GNN is 1-hop (user -> item x ) [item has no outgoing nodes]
        # Add edges in both directions 
        edge_index.append([u_idx, i_idx])
        edge_index.append([i_idx, u_idx])
    edge_index = torch.tensor(edge_index).t().contiguous()
    # Masks for users and items
    user_mask = torch.zeros(num_users + num_items, dtype=torch.bool)
    item_mask = torch.zeros(num_users + num_items, dtype=torch.bool)
    user_mask[:num_users] = True
    item_mask[num_users:] = True    # Create data object
    data = Data(x=x, edge_index=edge_index)
    data.user_mask = user_mask
    data.item_mask = item_mask
    # positive edge index
    all_sample_interactions = []
    for uid, iid in user_item_all_interactions:
        if iid in item_id_map:
            u_idx = user_id_map[uid]
            i_idx = item_id_map[iid]
            all_sample_interactions.append([u_idx, i_idx])
        else:
            print(f'{iid} not in item_id_map')
    data.all_sample_index = torch.tensor(all_sample_interactions, dtype = torch.long).t().contiguous()
    
    labels = []
    for uid, iid in user_item_all_interactions:
        if (uid, iid) in positive_interactions_set:
            labels.append(1)
        else:
            labels.append(0)
    data.labels = torch.tensor(labels, dtype=torch.float)
    data.edge_users = [user_id_map[uid] for uid, iid in user_item_all_interactions]
    data.edge_items = [item_id_map[iid] for uid, iid in user_item_all_interactions]
    
    return  data
print('Creating train graph')
train_data = prepare_graph(
    user_item_interactions = train_user_item_interactions.to_numpy(),
    user_item_all_interactions = train_user_item_all_interactions.to_numpy(),
    item_features = train_track_features,
    positive_interactions_set=train_positive_interactions_set
)
print('Creating test graph')
test_data = prepare_graph(
    user_item_interactions = test_user_item_interactions.to_numpy(),
    user_item_all_interactions = test_user_item_all_interactions.to_numpy(),
    item_features = test_track_features,
    positive_interactions_set=test_positive_interactions_set
    
)


Creating train graph
Number of users:  453
Number of items:  1981
Creating test graph
Number of users:  114
Number of items:  2132


In [97]:
test_data

Data(x=[2246, 768], edge_index=[2, 37846], user_mask=[2246], item_mask=[2246], all_sample_index=[2, 189230], labels=[189230], edge_users=[189230], edge_items=[189230])

In [ ]:
torch.save(train_data,os.path.join(data_dir, 'train_graph.pt'))
torch.save(test_data, os.path.join(data_dir, 'test_graph.pt'))

## Descriptive statistics

In [98]:
train_graph = torch.load(os.path.join(data_dir, 'train_graph.pt'))
test_graph = torch.load(os.path.join(data_dir, 'test_graph.pt'))

C:\Users\Gaurav Anand\AppData\Local\Temp\ipykernel_38532\1119227663.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_graph = torch.load(os.path.join(data_dir, 'trai

In [106]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.optim import Adam

# Define a simple GNN model
class SimpleGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleGNN, self).__init__()
        # Graph convolution layers
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First graph convolution + ReLU
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        # Second graph convolution
        x = self.conv2(x, edge_index)

        return x

# Instantiate the model, optimizer, and define a loss function
input_dim = train_data.x.shape[1]  # Feature dimension
hidden_dim = 64
output_dim = 1  # Binary classification (interaction or not)

model = SimpleGNN(input_dim, hidden_dim, output_dim)
optimizer = Adam(model.parameters(), lr=0.01)
loss_function = torch.nn.BCEWithLogitsLoss()

# Prepare training and testing datasets
train_loader = DataLoader([train_data], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data], batch_size=1, shuffle=False)

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)

        # Ensure output tensor dimensions align with expected indices
        if out.size(0) <= max(data.all_sample_index[0]) or out.size(0) <= max(data.all_sample_index[1]):
            print("Warning: Index out of range. Skipping this batch.")
            continue

        # Predicting edge existence using the all_sample_index
        edge_preds = out[data.all_sample_index[0], data.all_sample_index[1]].squeeze()
        labels = data.labels

        # Ensure we match the dimensions of edge_preds and labels
        if len(edge_preds) != len(labels):
            print("Warning: Mismatch between edge_preds and labels size. Skipping this batch.")
            continue

        loss = loss_function(edge_preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {total_loss:.4f}')

print("Training completed.")


IndexError: index 453 is out of bounds for dimension 1 with size 1

In [113]:
train_data

Data(x=[2434, 768], edge_index=[2, 124402], user_mask=[2434], item_mask=[2434], all_sample_index=[2, 622010], labels=[622010], edge_users=[622010], edge_items=[622010])

In [103]:
out

tensor([[   nan],
        [   nan],
        [0.0221],
        ...,
        [   nan],
        [   nan],
        [   nan]], grad_fn=<AddBackward0>)